# Spatial Error Model (SEM)

**Level**: Intermediate  
**Duration**: 120-150 minutes  
**Language**: English

---

## Objective

Understand the Spatial Error Model (SEM) and clearly differentiate it from SAR. Learn when spatial correlation is a "nuisance" rather than structural, estimate SEM via GMM and ML, and use LM tests to choose between SAR and SEM.

## 1. Introduction to SEM

### Spatial Correlation in Errors, Not Outcomes

**Model Specification**:
```
y = Xβ + α + u
u = λWu + ε
```

Where:
- **λ**: Spatial autoregressive parameter in ERROR term
- **Wu**: Spatial lag of error (spatially correlated shocks)
- **ε**: i.i.d. random error

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Add panelbox to path
panelbox_path = Path("/home/guhaase/projetos/panelbox")
sys.path.insert(0, str(panelbox_path))

# Load agricultural data
data_path = Path("../data/agriculture/agricultural_productivity.csv")
ag_data = pd.read_csv(data_path)

print("Dataset: Agricultural Productivity")
print("=" * 60)
print(ag_data.head(10))
print("\nShape:", ag_data.shape)
print("Years:", sorted(ag_data['year'].unique()))
print("Regions:", ag_data['region_id'].nunique())

## 2. Visualize Agricultural Yield Pattern

In [ ]:
# Load shapefile
shapefile_path = Path("../data/agriculture/agricultural_regions.shp")
ag_geo = gpd.read_file(shapefile_path)
ag_geo = ag_geo.merge(ag_data[ag_data['year'] == 2022], on='region_id')

# Map yield
fig, ax = plt.subplots(figsize=(12, 10))
ag_geo.plot(column='yield',
            cmap='YlGn',
            legend=True,
            ax=ax,
            edgecolor='black',
            linewidth=0.5,
            legend_kwds={'label': 'Yield (tons/ha)', 'shrink': 0.8})
ax.set_title('Agricultural Yield (Spatial Pattern)', fontsize=14, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.savefig('../outputs/figures/nb04_yield_map.png', dpi=300, bbox_inches='tight')
plt.show()

print("→ Look for spatial clustering (high-yield regions together)")

## 3. OLS with Spatially Correlated Errors

In [ ]:
from sklearn.linear_model import LinearRegression
from libpysal.weights import Queen

# Prepare data for 2022
ag_2022 = ag_data[ag_data['year'] == 2022].sort_values('region_id').reset_index(drop=True)

X_vars = ['fertilizer', 'rainfall', 'irrigation']
X = ag_2022[X_vars].values
y = ag_2022['yield'].values

# OLS estimation
ols = LinearRegression()
ols.fit(X, y)
residuals_ols = y - ols.predict(X)

print("OLS RESULTS")
print("=" * 60)
for var, coef in zip(X_vars, ols.coef_):
    print(f"{var:12s}: {coef:8.3f}")
print(f"Intercept: {ols.intercept_:8.3f}")
print("=" * 60)

## 4. Check Residual Spatial Autocorrelation

In [ ]:
from esda import Moran

# Create spatial weights
W = Queen.from_dataframe(ag_geo)
W.transform = 'r'

# Moran's I on residuals
moran_resid = Moran(residuals_ols, W)

print(f"\nMoran's I on OLS Residuals:")
print(f"  I = {moran_resid.I:.4f}")
print(f"  p-value = {moran_resid.p_sim:.4f}")

if moran_resid.p_sim < 0.05:
    print("  ⚠ Residuals are spatially autocorrelated!")
    print("  → OLS standard errors are WRONG")
    print("  → Hypothesis tests are invalid")
else:
    print("  ✓ No significant spatial autocorrelation")

## 5. Visualize Residual Pattern

In [ ]:
ag_geo['ols_resid'] = residuals_ols

fig, ax = plt.subplots(figsize=(12, 10))
ag_geo.plot(column='ols_resid',
            cmap='RdBu_r',
            legend=True,
            ax=ax,
            edgecolor='black',
            linewidth=0.5,
            legend_kwds={'label': 'OLS Residuals', 'shrink': 0.8})
ax.set_title('OLS Residuals (Spatially Clustered)', fontsize=14, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.savefig('../outputs/figures/nb04_ols_residuals.png', dpi=300, bbox_inches='tight')
plt.show()

print("→ Notice spatial clustering of positive (red) and negative (blue) residuals")

## 6. Estimating SEM with GMM

In [ ]:
# For SEM estimation, we'll use spreg which has robust implementation
try:
    from spreg import GM_Error
    
    # Prepare data
    y_array = y.reshape(-1, 1)
    X_array = np.column_stack([np.ones(len(y)), X])
    
    # Estimate SEM using GMM
    sem_gmm = GM_Error(
        y_array,
        X_array,
        w=W,
        name_y='yield',
        name_x=['const'] + X_vars,
        name_w='W',
        name_ds='Agricultural Data'
    )
    
    print("\nSEM GMM ESTIMATION")
    print("=" * 60)
    print(sem_gmm.summary)
    
except ImportError:
    print("Note: spreg not available. Using alternative estimation...")
    
    # Simple alternative: Use spatial Cochrane-Orcutt transformation
    W_array = W.full()[0]
    Wu = W_array @ residuals_ols
    lambda_hat = np.corrcoef(residuals_ols, Wu)[0, 1]
    
    # Transform variables
    I_lambdaW = np.eye(len(y)) - lambda_hat * W_array
    y_trans = I_lambdaW @ y
    X_trans = I_lambdaW @ X
    
    # Re-estimate OLS on transformed data
    ols_trans = LinearRegression()
    ols_trans.fit(X_trans, y_trans)
    
    print("\nSEM ESTIMATION (Cochrane-Orcutt)")
    print("=" * 60)
    print(f"Lambda (spatial error parameter): {lambda_hat:.4f}")
    print(f"\nCoefficients:")
    for var, coef in zip(X_vars, ols_trans.coef_):
        print(f"  {var:12s}: {coef:8.3f}")
    print(f"  Intercept: {ols_trans.intercept_:8.3f}")
    print("=" * 60)

## 7. LM Tests: SAR vs SEM vs OLS

In [ ]:
try:
    from spreg import OLS as OLS_spreg
    
    # Run OLS with spreg to get diagnostics
    ols_spreg = OLS_spreg(
        y_array,
        X_array,
        w=W,
        spat_diag=True,
        name_y='yield',
        name_x=['const'] + X_vars
    )
    
    print("LAGRANGE MULTIPLIER TESTS")
    print("=" * 70)
    print(f"{'Test':<25} {'Statistic':>12} {'p-value':>12} {'Decision':>15}")
    print("-" * 70)
    
    # Extract LM test results
    lm_tests = {
        'LM-lag': (ols_spreg.lm_lag[0], ols_spreg.lm_lag[1]),
        'LM-error': (ols_spreg.lm_error[0], ols_spreg.lm_error[1]),
        'Robust LM-lag': (ols_spreg.rlm_lag[0], ols_spreg.rlm_lag[1]),
        'Robust LM-error': (ols_spreg.rlm_error[0], ols_spreg.rlm_error[1])
    }
    
    for test_name, (stat, pval) in lm_tests.items():
        decision = 'Reject H0' if pval < 0.05 else 'Fail to reject'
        print(f"{test_name:<25} {stat:>12.3f} {pval:>12.4f} {decision:>15}")
    
    print("=" * 70)
    
except ImportError:
    print("Note: LM tests require spreg package")
    print("\nWe can use Moran's I as a simple alternative:")
    print(f"  Moran's I on residuals: {moran_resid.I:.4f} (p = {moran_resid.p_sim:.4f})")
    if moran_resid.p_sim < 0.05:
        print("  → Significant spatial autocorrelation detected")
        print("  → Consider SEM or SAR model")
    
    # Create mock LM tests for demonstration
    lm_tests = {
        'LM-lag': (2.45, 0.117),
        'LM-error': (8.92, 0.003),
        'Robust LM-lag': (0.15, 0.698),
        'Robust LM-error': (6.62, 0.010)
    }

## 8. Interpret LM Tests

In [ ]:
lm_lag_sig = lm_tests['LM-lag'][1] < 0.05
lm_error_sig = lm_tests['LM-error'][1] < 0.05

print("\nDECISION BASED ON LM TESTS:")
print("=" * 60)

if not lm_lag_sig and not lm_error_sig:
    print("→ Use OLS (no spatial dependence)")
elif lm_lag_sig and not lm_error_sig:
    print("→ Use SAR (spatial lag dependence)")
elif lm_error_sig and not lm_lag_sig:
    print("→ Use SEM (spatial error correlation)")
else:
    print("→ Both significant: Check Robust LM tests")
    
    robust_lag_sig = lm_tests['Robust LM-lag'][1] < 0.05
    robust_error_sig = lm_tests['Robust LM-error'][1] < 0.05
    
    if robust_lag_sig and not robust_error_sig:
        print("   → Robust LM-lag significant → Use SAR")
    elif robust_error_sig and not robust_lag_sig:
        print("   → Robust LM-error significant → Use SEM")
    else:
        print("   → Both robust significant → Use SDM or GNS (Notebook 05)")

print("=" * 60)

## 9. Visualize LM Test Results

In [ ]:
test_names = list(lm_tests.keys())
statistics = [lm_tests[t][0] for t in test_names]
pvalues = [lm_tests[t][1] for t in test_names]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Subplot 1: Statistics
colors = ['green' if p < 0.05 else 'gray' for p in pvalues]
axes[0].bar(range(len(test_names)), statistics, color=colors, alpha=0.7, edgecolor='black')
axes[0].set_xticks(range(len(test_names)))
axes[0].set_xticklabels(test_names, rotation=15, ha='right')
axes[0].set_ylabel('LM Statistic')
axes[0].set_title('LM Test Statistics', fontweight='bold')
axes[0].grid(True, axis='y', alpha=0.3)

# Subplot 2: p-values
axes[1].bar(range(len(test_names)), pvalues, color=colors, alpha=0.7, edgecolor='black')
axes[1].axhline(0.05, color='red', linestyle='--', linewidth=2, label='α = 0.05')
axes[1].set_xticks(range(len(test_names)))
axes[1].set_xticklabels(test_names, rotation=15, ha='right')
axes[1].set_ylabel('p-value')
axes[1].set_title('LM Test p-values', fontweight='bold')
axes[1].legend()
axes[1].grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/figures/nb04_lm_tests.png', dpi=300, bbox_inches='tight')
plt.show()

print("→ Green bars: Reject H0 (spatial dependence present)")
print("→ Gray bars: Fail to reject H0 (no evidence of spatial dependence)")

## 10. Summary

### Key Takeaways

1. ✓ **SEM**: Spatial correlation in ERRORS (nuisance, not structural)
2. ✓ **λ > 0**: Omitted factors are spatially clustered
3. ✓ **Coefficients** similar to OLS, but SEs corrected
4. ✓ **Use LM tests** to choose SAR vs SEM
5. ✓ **GMM and ML** both estimate SEM
6. ✓ **SAR and SEM** are NOT interchangeable!